<a href="https://colab.research.google.com/github/sreent/machine-learning/blob/main/Linear%20Regression/Stock%20Price%20Prediction%20Case%20Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Study: Stock Price Prediction with Linear Regression

## The Challenge: Building a Unified Model Across Different Stocks

**Business Problem:**  
We want to predict stock prices **4 weeks ahead** for FTSE 100 companies using historical data. However, we face a critical challenge:

- **Stock AAF.L** trades around **£70**
- **Stock WTB.L** trades around **£3,000**

**Why is this a problem?**

If we naively use raw prices as both features and targets:
- A £5 prediction error on AAF.L (£70) = **7% error** 😱
- A £5 prediction error on WTB.L (£3,000) = **0.17% error** ✅

The model would optimize for expensive stocks and perform terribly on cheaper ones!

**The Solution: Think Like a Quant Trader**

Professional traders don't care about absolute price movements - they care about **percentage returns**:

$$\text{Return} = \frac{\text{Price}_{today} - \text{Price}_{yesterday}}{\text{Price}_{yesterday}}$$

- A **5% gain** is meaningful whether on a £70 or £3,000 stock
- Returns are **scale-invariant** - perfect for a unified model!
- Returns are **additive** over time (log returns)

**What We'll Build:**
1. Load ~580K rows of FTSE 100 daily OHLC data (2019-2025)
2. Engineer financial features: returns, volatility, momentum, moving averages
3. Create lagged features (past returns predict future returns)
4. Handle time-series data properly (no future data leakage!)
5. Build a unified linear regression model predicting **4-week returns**
6. Evaluate performance and understand limitations

## Learning Objectives

By the end of this case study, you will:

✅ Understand why **returns** (not prices) are used in financial ML  
✅ Learn proper **time-series cross-validation** to avoid lookahead bias  
✅ Engineer **domain-specific features** for stock prediction  
✅ Handle **multiple entities** (100 stocks) in a unified model  
✅ Interpret model performance in financial context  
✅ Understand the **challenges and limitations** of linear models for finance

## Step 1: Import Libraries and Load Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("Libraries imported successfully!")

In [ ]:
# Load the FTSE 100 dataset
df = pd.read_csv('ftse100.csv', parse_dates=['date'])

print(f"Dataset shape: {df.shape}")
print(f"\nDate range: {df['date'].min()} to {df['date'].max()}")
print(f"Number of unique tickers: {df['ticker'].nunique()}")
print(f"\nFirst few rows:")
df.head(10)

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())
print(f"\nBasic statistics:")
df.describe()

## Step 2: Exploratory Data Analysis - The Price Scale Problem

In [ ]:
# Analyze price ranges across different stocks
price_stats = df.groupby('ticker')['close'].agg(['min', 'mean', 'max', 'std']).sort_values('mean')

print("Price statistics by ticker (sorted by mean price):")
print("\nCheapest stocks:")
print(price_stats.head(5))
print("\nMost expensive stocks:")
print(price_stats.tail(5))

# Visualize the problem
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Price range distribution
axes[0].hist(price_stats['mean'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Mean Close Price (£)', fontsize=12)
axes[0].set_ylabel('Number of Stocks', fontsize=12)
axes[0].set_title('Distribution of Average Stock Prices\n(Why we can\'t use raw prices!)', fontsize=14)
axes[0].axvline(price_stats['mean'].median(), color='red', linestyle='--', 
               linewidth=2, label=f'Median: £{price_stats["mean"].median():.2f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Example time series for cheap vs expensive stocks
cheap_stock = price_stats.index[0]  # Cheapest
expensive_stock = price_stats.index[-1]  # Most expensive

cheap_data = df[df['ticker'] == cheap_stock].set_index('date')['close']
expensive_data = df[df['ticker'] == expensive_stock].set_index('date')['close']

ax2 = axes[1]
ax2_twin = ax2.twinx()

ax2.plot(cheap_data.index, cheap_data.values, 'b-', linewidth=1.5, label=f'{cheap_stock} (Cheap)', alpha=0.7)
ax2_twin.plot(expensive_data.index, expensive_data.values, 'r-', linewidth=1.5, label=f'{expensive_stock} (Expensive)', alpha=0.7)

ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel(f'{cheap_stock} Price (£)', color='b', fontsize=12)
ax2_twin.set_ylabel(f'{expensive_stock} Price (£)', color='r', fontsize=12)
ax2.set_title('Different Price Scales = Need Returns!', fontsize=14)
ax2.tick_params(axis='y', labelcolor='b')
ax2_twin.tick_params(axis='y', labelcolor='r')
ax2.grid(True, alpha=0.3)

# Combine legends
lines1, labels1 = ax2.get_legend_handles_labels()
lines2, labels2 = ax2_twin.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.show()

print(f"\n💡 Key Insight: Prices range from £{price_stats['mean'].min():.2f} to £{price_stats['mean'].max():.2f}")
print(f"   This {price_stats['mean'].max() / price_stats['mean'].min():.0f}x difference makes raw prices unusable!")

## Step 3: Feature Engineering - The Solution

### Understanding Financial Returns

**Simple Return (what traders report):**
$$r_t = \frac{P_t - P_{t-1}}{P_{t-1}} = \frac{P_t}{P_{t-1}} - 1$$

**Log Return (what quants use):**
$$r_t = \ln\left(\frac{P_t}{P_{t-1}}\right) = \ln(P_t) - \ln(P_{t-1})$$

**Why log returns?**
1. **Time-additivity**: Total return over n periods = sum of daily log returns
2. **Symmetry**: +10% gain and -10% loss have similar magnitudes in log space
3. **Statistical properties**: More normally distributed (better for linear models)
4. **Numerical stability**: No division by potentially small numbers

For small returns (< 10%), they're approximately equal: $\ln(1 + r) \approx r$

In [ ]:
def engineer_features(df):
    """
    Engineer financial features from OHLC data.
    All features are designed to be scale-invariant!
    """
    # Sort by ticker and date to ensure proper time series order
    df = df.sort_values(['ticker', 'date']).reset_index(drop=True)
    
    # Group by ticker for time-series operations
    grouped = df.groupby('ticker')
    
    # ========================================
    # 1. RETURNS (our main features and target)
    # ========================================
    # Daily log return (today's feature)
    df['return_1d'] = grouped['close'].transform(lambda x: np.log(x / x.shift(1)))
    
    # Intraday return (open to close)
    df['return_intraday'] = np.log(df['close'] / df['open'])
    
    # Overnight return (previous close to today's open)
    df['return_overnight'] = grouped.apply(lambda x: np.log(x['open'] / x['close'].shift(1))).reset_index(level=0, drop=True)
    
    # ========================================
    # 2. LAGGED RETURNS (past performance)
    # ========================================
    # Past 5 days of returns
    for lag in [1, 2, 3, 4, 5]:
        df[f'return_lag_{lag}d'] = grouped['return_1d'].shift(lag)
    
    # Past week average return
    df['return_mean_5d'] = grouped['return_1d'].transform(lambda x: x.shift(1).rolling(5).mean())
    
    # Past 4-week return (starting point for our prediction)
    df['return_4w'] = grouped['close'].transform(lambda x: np.log(x / x.shift(20)))
    
    # ========================================
    # 3. VOLATILITY (risk measure)
    # ========================================
    # 5-day rolling standard deviation of returns
    df['volatility_5d'] = grouped['return_1d'].transform(lambda x: x.shift(1).rolling(5).std())
    
    # Intraday range (high-low volatility proxy)
    df['range_hl'] = (df['high'] - df['low']) / df['close']
    
    # ========================================
    # 4. MOMENTUM INDICATORS
    # ========================================
    # 5-day momentum (comparing to 5 days ago)
    df['momentum_5d'] = grouped['close'].transform(lambda x: (x / x.shift(5)) - 1)
    
    # 10-day momentum
    df['momentum_10d'] = grouped['close'].transform(lambda x: (x / x.shift(10)) - 1)
    
    # ========================================
    # 5. MOVING AVERAGES (trend indicators)
    # ========================================
    # Distance from 20-day moving average (normalized)
    df['ma_20d'] = grouped['close'].transform(lambda x: x.rolling(20).mean())
    df['dist_from_ma20'] = (df['close'] - df['ma_20d']) / df['ma_20d']
    
    # ========================================
    # 6. TARGET: 4-WEEK FUTURE RETURN
    # ========================================
    # This is what we want to predict!
    # Shift by -20 trading days (4 weeks * 5 days/week)
    df['target_return_4w'] = grouped['close'].transform(lambda x: np.log(x.shift(-20) / x))
    
    return df

print("Engineering features...")
df_features = engineer_features(df.copy())
print(f"✅ Created {len(df_features.columns) - len(df.columns)} new features")
print(f"\nNew columns: {list(df_features.columns[len(df.columns):])}")

In [ ]:
# Inspect the engineered features
print("Sample of engineered features:")
display_cols = ['date', 'ticker', 'close', 'return_1d', 'return_lag_1d', 'volatility_5d', 
                'momentum_5d', 'dist_from_ma20', 'target_return_4w']
df_features[display_cols].head(30)

## Step 4: Visualize Returns vs Prices

In [ ]:
# Compare distributions: prices vs returns
# Pick two stocks with very different price levels
cheap_ticker = price_stats.index[0]
expensive_ticker = price_stats.index[-1]

cheap_df = df_features[df_features['ticker'] == cheap_ticker].dropna()
expensive_df = df_features[df_features['ticker'] == expensive_ticker].dropna()

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Row 1: Prices
axes[0, 0].hist(cheap_df['close'], bins=50, alpha=0.7, edgecolor='black', label=cheap_ticker)
axes[0, 0].set_xlabel('Close Price (£)', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title(f'{cheap_ticker} Price Distribution', fontsize=14)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].hist(expensive_df['close'], bins=50, alpha=0.7, color='orange', edgecolor='black', label=expensive_ticker)
axes[0, 1].set_xlabel('Close Price (£)', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title(f'{expensive_ticker} Price Distribution', fontsize=14)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Row 2: Returns (COMPARABLE!)
axes[1, 0].hist(cheap_df['return_1d'].dropna() * 100, bins=50, alpha=0.7, edgecolor='black', label=cheap_ticker)
axes[1, 0].set_xlabel('Daily Return (%)', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].set_title(f'{cheap_ticker} Return Distribution\n(Now comparable!)', fontsize=14)
axes[1, 0].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlim(-10, 10)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].hist(expensive_df['return_1d'].dropna() * 100, bins=50, alpha=0.7, color='orange', edgecolor='black', label=expensive_ticker)
axes[1, 1].set_xlabel('Daily Return (%)', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title(f'{expensive_ticker} Return Distribution\n(Similar scale!)', fontsize=14)
axes[1, 1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1, 1].set_xlim(-10, 10)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💡 Key Insight: Returns put both stocks on the SAME scale!")
print(f"   {cheap_ticker} return std: {cheap_df['return_1d'].std()*100:.2f}%")
print(f"   {expensive_ticker} return std: {expensive_df['return_1d'].std()*100:.2f}%")
print(f"   These are comparable, unlike prices!")

## Step 5: Prepare Data for Modeling

In [ ]:
# Define feature columns (what we'll use to predict)
feature_cols = [
    # Lagged returns
    'return_lag_1d', 'return_lag_2d', 'return_lag_3d', 'return_lag_4d', 'return_lag_5d',
    'return_mean_5d',
    
    # Volatility
    'volatility_5d',
    'range_hl',
    
    # Momentum
    'momentum_5d',
    'momentum_10d',
    
    # Moving averages
    'dist_from_ma20',
    
    # Intraday behavior
    'return_intraday',
    'return_overnight'
]

target_col = 'target_return_4w'

# Remove rows with missing values (due to lagging and rolling windows)
df_clean = df_features[feature_cols + [target_col, 'date', 'ticker', 'close']].dropna()

print(f"Original dataset: {len(df_features)} rows")
print(f"After removing NaN: {len(df_clean)} rows ({len(df_clean)/len(df_features)*100:.1f}% retained)")
print(f"\nFeatures: {len(feature_cols)}")
print(f"Target: {target_col}")
print(f"\nDate range: {df_clean['date'].min()} to {df_clean['date'].max()}")

## Step 6: Time-Series Split (CRITICAL!)

### ⚠️ Why we CAN'T use random train/test split for time series:

**Random split = Future data leakage!**

Imagine it's January 2022, and you randomly select:
- Training: Some data from 2020, 2021, **2022, 2023**
- Testing: Some data from 2020, **2021**, 2022

Your model is using **future data** (2023) to predict the **past** (2021)! This would never work in real trading.

**Correct approach: Time-based split**
- Train: 2019-07-01 to 2023-12-31 (4.5 years)
- Validation: 2024-01-01 to 2024-12-31 (1 year)
- Test: 2025-01-01 onwards (most recent)

This simulates real-world deployment: train on past, predict future!

In [ ]:
# Time-based split
train_end = '2023-12-31'
val_end = '2024-12-31'

train_df = df_clean[df_clean['date'] <= train_end]
val_df = df_clean[(df_clean['date'] > train_end) & (df_clean['date'] <= val_end)]
test_df = df_clean[df_clean['date'] > val_end]

print("Time-Series Split:")
print("="*70)
print(f"Training:   {train_df['date'].min()} to {train_df['date'].max()} | {len(train_df):,} rows")
print(f"Validation: {val_df['date'].min()} to {val_df['date'].max()} | {len(val_df):,} rows")
print(f"Test:       {test_df['date'].min()} to {test_df['date'].max()} | {len(test_df):,} rows")
print("="*70)

# Extract features and targets
X_train = train_df[feature_cols].values
y_train = train_df[target_col].values

X_val = val_df[feature_cols].values
y_val = val_df[target_col].values

X_test = test_df[feature_cols].values
y_test = test_df[target_col].values

print(f"\nX_train shape: {X_train.shape}")
print(f"X_val shape:   {X_val.shape}")
print(f"X_test shape:  {X_test.shape}")

## Step 7: Feature Scaling

In [ ]:
# Standardize features using training data statistics
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling completed!")
print(f"\nTraining set feature statistics (after scaling):")
print(f"  Mean: {X_train_scaled.mean(axis=0).round(6)}")
print(f"  Std:  {X_train_scaled.std(axis=0).round(6)}")
print(f"\n✅ All features now have mean ≈ 0 and std ≈ 1")

## Step 8: Train Linear Regression Model

In [ ]:
# Train the model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Make predictions
y_train_pred = model.predict(X_train_scaled)
y_val_pred = model.predict(X_val_scaled)
y_test_pred = model.predict(X_test_scaled)

print("Model trained successfully!")
print(f"\nModel parameters:")
print(f"  Intercept: {model.intercept_:.6f}")
print(f"  Number of coefficients: {len(model.coef_)}")

## Step 9: Evaluate Performance

In [ ]:
def evaluate_model(y_true, y_pred, set_name=""):
    """
    Evaluate regression model with financial metrics.
    """
    # Standard metrics
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Financial metrics (convert log returns to percentage)
    error_pct = np.abs(y_true - y_pred) * 100  # Absolute error in %
    
    print(f"\n{set_name} Performance:")
    print("="*50)
    print(f"  R² Score:          {r2:.4f}")
    print(f"  RMSE (log return): {rmse:.4f}")
    print(f"  MAE (log return):  {mae:.4f}")
    print(f"\n  RMSE (%):          {rmse*100:.2f}%")
    print(f"  MAE (%):           {mae*100:.2f}%")
    print(f"  Mean Abs Error:    {error_pct.mean():.2f}%")
    print(f"  Median Abs Error:  {np.median(error_pct):.2f}%")
    print("="*50)
    
    return {
        'r2': r2,
        'rmse': rmse,
        'mae': mae,
        'rmse_pct': rmse * 100,
        'mae_pct': mae * 100
    }

# Evaluate on all sets
train_metrics = evaluate_model(y_train, y_train_pred, "TRAINING")
val_metrics = evaluate_model(y_val, y_val_pred, "VALIDATION")
test_metrics = evaluate_model(y_test, y_test_pred, "TEST")

# Summary comparison
print("\n" + "="*70)
print("SUMMARY COMPARISON")
print("="*70)
print(f"{'Metric':<20} {'Train':>12} {'Validation':>12} {'Test':>12}")
print("-"*70)
print(f"{'R² Score':<20} {train_metrics['r2']:>12.4f} {val_metrics['r2']:>12.4f} {test_metrics['r2']:>12.4f}")
print(f"{'RMSE (%)':<20} {train_metrics['rmse_pct']:>12.2f} {val_metrics['rmse_pct']:>12.2f} {test_metrics['rmse_pct']:>12.2f}")
print(f"{'MAE (%)':<20} {train_metrics['mae_pct']:>12.2f} {val_metrics['mae_pct']:>12.2f} {test_metrics['mae_pct']:>12.2f}")
print("="*70)

## Step 10: Feature Importance Analysis

In [ ]:
# Analyze feature importance (from standardized coefficients)
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 8))
colors = ['green' if x > 0 else 'red' for x in feature_importance['Coefficient']]
plt.barh(feature_importance['Feature'], feature_importance['Coefficient'], color=colors, edgecolor='black')
plt.xlabel('Standardized Coefficient', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Feature Importance for 4-Week Return Prediction', fontsize=14)
plt.axvline(x=0, color='black', linewidth=1)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nTop 5 Most Important Features:")
print(feature_importance.head())

print("\n💡 Interpretation:")
print("   Green = Positive coefficient (higher feature → higher future return)")
print("   Red   = Negative coefficient (higher feature → lower future return)")
print("   Magnitude = Strength of relationship")

## Step 11: Visualize Predictions

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Predicted vs Actual (Validation Set)
axes[0, 0].scatter(y_val * 100, y_val_pred * 100, alpha=0.3, s=10)
axes[0, 0].plot([y_val.min()*100, y_val.max()*100], [y_val.min()*100, y_val.max()*100], 
               'r--', linewidth=2, label='Perfect Prediction')
axes[0, 0].set_xlabel('Actual 4-Week Return (%)', fontsize=12)
axes[0, 0].set_ylabel('Predicted 4-Week Return (%)', fontsize=12)
axes[0, 0].set_title(f'Validation Set: Predictions vs Actual\nR² = {val_metrics["r2"]:.4f}', fontsize=14)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(0, color='gray', linewidth=0.5)
axes[0, 0].axvline(0, color='gray', linewidth=0.5)

# Plot 2: Residual Distribution
residuals_val = (y_val - y_val_pred) * 100
axes[0, 1].hist(residuals_val, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Prediction Error (%)', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title(f'Distribution of Prediction Errors\nMean: {residuals_val.mean():.2f}% | Std: {residuals_val.std():.2f}%', fontsize=14)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Error vs Predicted Return
axes[1, 0].scatter(y_val_pred * 100, residuals_val, alpha=0.3, s=10)
axes[1, 0].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Predicted Return (%)', fontsize=12)
axes[1, 0].set_ylabel('Prediction Error (%)', fontsize=12)
axes[1, 0].set_title('Residual Plot: Check for Patterns', fontsize=14)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Cumulative Error Over Time
val_df_copy = val_df.copy()
val_df_copy['predicted'] = y_val_pred
val_df_copy['error'] = np.abs(y_val - y_val_pred) * 100
error_by_date = val_df_copy.groupby('date')['error'].mean().sort_index()
axes[1, 1].plot(error_by_date.index, error_by_date.values, linewidth=1.5)
axes[1, 1].axhline(error_by_date.mean(), color='red', linestyle='--', linewidth=2, 
                  label=f'Mean: {error_by_date.mean():.2f}%')
axes[1, 1].set_xlabel('Date', fontsize=12)
axes[1, 1].set_ylabel('Mean Absolute Error (%)', fontsize=12)
axes[1, 1].set_title('Prediction Error Over Time', fontsize=14)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Step 12: Directional Accuracy (Trading Perspective)

In [ ]:
# For trading, direction matters more than exact magnitude!
# Can we predict if stock will go UP or DOWN?

def directional_accuracy(y_true, y_pred):
    """
    Calculate what percentage of time we correctly predict direction.
    """
    # Convert to up/down signals
    actual_direction = (y_true > 0).astype(int)  # 1 if up, 0 if down
    pred_direction = (y_pred > 0).astype(int)
    
    accuracy = (actual_direction == pred_direction).mean()
    
    return accuracy, actual_direction, pred_direction

# Calculate directional accuracy
train_dir_acc, _, _ = directional_accuracy(y_train, y_train_pred)
val_dir_acc, val_actual_dir, val_pred_dir = directional_accuracy(y_val, y_val_pred)
test_dir_acc, _, _ = directional_accuracy(y_test, y_test_pred)

print("\n" + "="*70)
print("DIRECTIONAL ACCURACY (Can we predict UP vs DOWN?)")
print("="*70)
print(f"Training:   {train_dir_acc*100:.2f}%")
print(f"Validation: {val_dir_acc*100:.2f}%")
print(f"Test:       {test_dir_acc*100:.2f}%")
print("="*70)
print(f"\nBaseline (random guessing): 50.00%")
print(f"Our model beats random: {val_dir_acc > 0.5}")

# Confusion matrix for validation set
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(val_actual_dir, val_pred_dir)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Predicted DOWN', 'Predicted UP'],
            yticklabels=['Actual DOWN', 'Actual UP'],
            cbar_kws={'label': 'Count'})
plt.ylabel('Actual Direction', fontsize=12)
plt.xlabel('Predicted Direction', fontsize=12)
plt.title(f'Confusion Matrix: Direction Prediction\nAccuracy: {val_dir_acc*100:.2f}%', fontsize=14)
plt.tight_layout()
plt.show()

print("\nClassification Report (Direction):")
print(classification_report(val_actual_dir, val_pred_dir, 
                          target_names=['DOWN', 'UP']))

## Step 13: Case Study - Individual Stock Predictions

In [ ]:
# Let's examine predictions for specific stocks
# Pick 3 stocks: cheap, medium, expensive
sample_tickers = [price_stats.index[0], price_stats.index[len(price_stats)//2], price_stats.index[-1]]

fig, axes = plt.subplots(3, 1, figsize=(15, 12))

for idx, ticker in enumerate(sample_tickers):
    # Get validation data for this ticker
    ticker_val = val_df[val_df['ticker'] == ticker].copy()
    ticker_indices = val_df[val_df['ticker'] == ticker].index
    ticker_val['predicted'] = y_val_pred[val_df['ticker'] == ticker]
    
    if len(ticker_val) > 0:
        # Plot actual vs predicted returns over time
        ax = axes[idx]
        ax.plot(ticker_val['date'], ticker_val['target_return_4w'] * 100, 
               'b-', linewidth=2, label='Actual 4W Return', alpha=0.7)
        ax.plot(ticker_val['date'], ticker_val['predicted'] * 100, 
               'r--', linewidth=2, label='Predicted 4W Return', alpha=0.7)
        ax.axhline(0, color='black', linewidth=0.5)
        ax.set_xlabel('Date', fontsize=11)
        ax.set_ylabel('4-Week Return (%)', fontsize=11)
        
        # Calculate R² for this ticker
        ticker_r2 = r2_score(ticker_val['target_return_4w'], ticker_val['predicted'])
        avg_price = price_stats.loc[ticker, 'mean']
        
        ax.set_title(f'{ticker} (Avg Price: £{avg_price:.2f}) | R² = {ticker_r2:.4f}', fontsize=13)
        ax.legend(loc='upper left')
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n💡 Key Insight: The model works across ALL price levels!")
print("   Returns normalize the different scales, enabling a unified model.")

## Step 14: Understanding Model Limitations

### Why Stock Prediction is Hard (Even with ML)

Our model's R² is likely **low** (< 0.05). This is **normal** for stock prediction! Here's why:

#### 1. **Efficient Market Hypothesis (EMH)**
- Markets are highly efficient
- Past prices already incorporate all historical information
- If prediction were easy, everyone would do it and eliminate the opportunity

#### 2. **Noise-to-Signal Ratio**
Stock returns = Predictable signal + Random noise

$$r_t = \underbrace{\mu + \beta_1 x_1 + ... + \beta_n x_n}_{\text{Signal (tiny)}} + \underbrace{\epsilon_t}_{\text{Noise (huge)}}$$

The noise dominates! Even a "good" R² of 0.01 means your model explains 1% of variance.

#### 3. **Linear Regression Limitations**
- Assumes linear relationships (markets are non-linear)
- Can't capture regime changes (bull markets, crashes, volatility spikes)
- Ignores cross-stock interactions
- No consideration of external events (news, earnings, macro data)

#### 4. **What Professional Quants Do Differently**
- Use **hundreds of features** (alternative data, news sentiment, options flow)
- Employ **ensemble models** (Random Forests, Gradient Boosting, Neural Networks)
- Implement **regime detection** (separate models for different market conditions)
- Focus on **directional accuracy** and **risk-adjusted returns**, not R²
- Trade **many stocks** to diversify away idiosyncratic risk
- Operate at **higher frequencies** (daily, hourly, minute-level)

#### 5. **The Real Benchmark**
Even a **slight** edge (51% directional accuracy instead of 50%) can be profitable with:
- Proper risk management
- Position sizing
- Transaction cost optimization
- Portfolio diversification

## Key Takeaways

### ✅ What We Learned

1. **Scale Invariance is Critical**
   - Raw prices vary 100x across stocks (£30 to £3,000)
   - Returns normalize this: both cheap and expensive stocks have ~2% daily volatility
   - Unified model trains on all stocks equally

2. **Time-Series Require Special Handling**
   - Never use random train/test splits (causes future data leakage)
   - Always split chronologically: train on past, test on future
   - Lagged features create dependencies that must be respected

3. **Domain Knowledge Matters**
   - Financial features (momentum, volatility, moving averages) are industry standard
   - Feature engineering dramatically improves over raw prices
   - Understanding the domain helps interpret model limitations

4. **Linear Regression for Finance**
   - **Strengths**: Fast, interpretable, works across all stocks
   - **Weaknesses**: Can't capture non-linearities, market regimes, or external shocks
   - **Reality**: Low R² is expected and doesn't mean failure

5. **Prediction vs Trading**
   - Directional accuracy matters more than exact values
   - Even 51% accuracy can be profitable with proper risk management
   - Real trading requires transaction costs, slippage, and position sizing

### 🚀 Extensions to Explore

1. **Better Models**: Try Ridge/Lasso regression, Random Forests, XGBoost
2. **More Features**: Add sector indicators, market-wide features, volatility indices
3. **Different Targets**: Predict volatility, drawdowns, or classification (up/down)
4. **Walk-Forward Analysis**: Retrain model periodically to adapt to changing markets
5. **Portfolio Optimization**: Use predictions to construct long/short portfolios

### 📚 Key Concepts Applied

| Concept | How We Used It |
|---------|---------------|
| **Linear Regression** | Unified model across 100 stocks |
| **Feature Engineering** | Returns, lagged features, momentum, volatility |
| **Feature Scaling** | StandardScaler on all features |
| **Time-Series Split** | Chronological train/val/test |
| **Model Evaluation** | R², RMSE, MAE, directional accuracy |
| **Domain Knowledge** | Financial returns, trading metrics |

### ⚠️ Important Warnings

**DO NOT** use this model for real trading without:
- Understanding market microstructure (bid-ask spreads, slippage)
- Implementing proper risk management (stop losses, position sizing)
- Accounting for transaction costs (commissions, taxes)
- Testing on more comprehensive data (multiple market cycles)
- Consulting with financial professionals

**This is for educational purposes only!**

## Bonus: Compare Against Naive Baselines

In [ ]:
# How does our model compare to simple baselines?

# Baseline 1: Always predict 0% return (market-neutral)
y_val_baseline_zero = np.zeros_like(y_val)

# Baseline 2: Predict mean historical return
y_val_baseline_mean = np.full_like(y_val, y_train.mean())

# Baseline 3: Momentum - assume next 4 weeks = last 4 weeks
y_val_baseline_momentum = val_df['return_4w'].values

# Evaluate baselines
print("\n" + "="*80)
print("COMPARISON: Our Model vs Simple Baselines")
print("="*80)

models = [
    ('Baseline: Always 0%', y_val_baseline_zero),
    ('Baseline: Mean Return', y_val_baseline_mean),
    ('Baseline: Momentum', y_val_baseline_momentum),
    ('Our Linear Regression', y_val_pred)
]

results = []
for name, preds in models:
    r2 = r2_score(y_val, preds)
    rmse = np.sqrt(mean_squared_error(y_val, preds)) * 100
    mae = mean_absolute_error(y_val, preds) * 100
    dir_acc, _, _ = directional_accuracy(y_val, preds)
    
    results.append({
        'Model': name,
        'R²': r2,
        'RMSE (%)': rmse,
        'MAE (%)': mae,
        'Dir. Acc (%)': dir_acc * 100
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))
print("="*80)

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: R² comparison
axes[0].bar(range(len(results_df)), results_df['R²'], 
           color=['gray', 'gray', 'gray', 'green'], edgecolor='black')
axes[0].set_xticks(range(len(results_df)))
axes[0].set_xticklabels(results_df['Model'], rotation=45, ha='right')
axes[0].set_ylabel('R² Score', fontsize=12)
axes[0].set_title('Model Comparison: R² Score', fontsize=14)
axes[0].axhline(0, color='red', linestyle='--', linewidth=1)
axes[0].grid(True, alpha=0.3, axis='y')

# Plot 2: Directional accuracy
axes[1].bar(range(len(results_df)), results_df['Dir. Acc (%)'], 
           color=['gray', 'gray', 'gray', 'green'], edgecolor='black')
axes[1].set_xticks(range(len(results_df)))
axes[1].set_xticklabels(results_df['Model'], rotation=45, ha='right')
axes[1].set_ylabel('Directional Accuracy (%)', fontsize=12)
axes[1].set_title('Model Comparison: Direction Prediction', fontsize=14)
axes[1].axhline(50, color='red', linestyle='--', linewidth=2, label='Random Guess')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n💡 Key Insight: Even if R² seems low, beating simple baselines is valuable!")
print("   Financial markets are noisy - any edge over random is significant.")

## Summary

In this case study, we tackled a real-world financial ML problem:

1. ✅ Identified the **scale problem** with stock prices
2. ✅ Solved it using **percentage returns** instead of absolute prices
3. ✅ Engineered **domain-specific features** (momentum, volatility, moving averages)
4. ✅ Properly handled **time-series data** (chronological splits, no lookahead bias)
5. ✅ Built a **unified linear regression model** for 100 different stocks
6. ✅ Evaluated with **financial metrics** (directional accuracy, risk-adjusted performance)
7. ✅ Understood **model limitations** and real-world challenges

**Most importantly**: We learned that **feature engineering and domain knowledge matter more than algorithm complexity** in many real-world problems!

Linear Regression, while simple, taught us:
- The importance of proper data transformation
- How to handle multi-entity datasets
- Why time-series require special treatment
- That "low R²" doesn't mean "bad model" in noisy domains

These lessons apply far beyond stock prediction - to demand forecasting, sensor data, medical time series, and more!